Load modules

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from PV import MyPV

검증 모듈

In [2]:
pv = MyPV(excel_file='./INFO_N.xlsx', fillNa="^", i=0.025)

Sample건

In [3]:
pv.MyCalculation(sex=1, x=40, n=5, m =5, coverageKey="LB0918", AMT = 1000000)

(77, 135, [0, 208, 320, 326, 220, 0])

In [4]:
pv.getSymbolTable()

,lx(0),Ex(0),lx(1),Ex(1),Bx(1),Cx(1),Mx(1),SUMx,l'x,Gx,Dx,D'x,Nx,N'x
0,100000.000000,0.000721,100000.000000,0.000721,0.000721,71.215304,544.313500,435.086693,100000.000000,0.000721,100000.000000,100000.000000,563407.748224,563407.748224
1,99927.900000,0.000821,99927.900000,0.000821,0.000821,79.057690,473.098196,363.871389,99927.900000,0.000821,97490.634146,97490.634146,463407.748224,463407.748224
2,99845.859194,0.000929,99845.859194,0.000929,0.000929,87.203940,394.040506,284.813700,99845.859194,0.000929,95034.726181,95034.726181,365917.114078,365917.114078
3,99753.102391,0.001040,99753.102391,0.001040,0.001040,95.153819,306.836566,197.609759,99753.102391,0.001040,92630.672118,92630.672118,270882.387897,270882.387897
4,99649.359164,0.001149,99649.359164,0.001149,0.001149,102.455941,211.682747,102.455941,99649.359164,0.001149,90277.401189,90277.401189,178251.715779,178251.715779
5,99534.862051,0.001257,99534.862051,0.001257,0.001257,109.226807,109.226807,0.000000,99534.862051,0.001257,87974.314590,87974.314590,87974.314590,87974.314590


대상건 테이블 읽기

In [5]:
df_check = pd.read_csv('Preprocessed.csv', encoding="cp949")
print(df_check.shape)
df_check.head()

(16580, 21)


,담보코드,보험기간,납입기간,보험기간구분,납입주기,성별,연령,상해급수,운전자급수,가입금액,...,Pr,G,V(0),V(1),V(2),V(3),V(4),V(5),신계약비,표준해약공제액
0,LB7408,5,5,A080,412,1,30,^,^,1000000,...,5,14,0,15,25,26,18,0,388,42.761583
1,LB7408,5,5,A080,412,1,31,^,^,1000000,...,6,15,0,20,30,31,23,0,391,49.220016
2,LB7408,5,5,A080,412,1,32,^,^,1000000,...,7,16,0,20,32,34,22,0,394,56.378872
3,LB7408,5,5,A080,412,1,33,^,^,1000000,...,8,18,0,23,38,38,29,0,397,64.794548
4,LB7408,5,5,A080,412,1,34,^,^,1000000,...,9,19,0,27,40,44,30,0,401,73.878022


In [6]:
print(df_check.columns)

Index(['담보코드', '보험기간', '납입기간', '보험기간구분', '납입주기', '성별', '연령', '상해급수', '운전자급수',
       '가입금액', 'NP', 'Pr', 'G', 'V(0)', 'V(1)', 'V(2)', 'V(3)', 'V(4)', 'V(5)',
       '신계약비', '표준해약공제액'],
      dtype='object')


In [7]:
df_check = df_check[['담보코드', '보험기간', '납입기간', '성별', '연령', '상해급수', '운전자급수', '가입금액', \
    'NP', 'G','V(0)','V(1)','V(2)','V(3)','V(4)','V(5)','신계약비', '표준해약공제액']]
print(df_check.shape)
df_check.head()

(16580, 18)


,담보코드,보험기간,납입기간,성별,연령,상해급수,운전자급수,가입금액,NP,G,V(0),V(1),V(2),V(3),V(4),V(5),신계약비,표준해약공제액
0,LB7408,5,5,1,30,^,^,1000000,5,14,0,15,25,26,18,0,388,42.761583
1,LB7408,5,5,1,31,^,^,1000000,6,15,0,20,30,31,23,0,391,49.220016
2,LB7408,5,5,1,32,^,^,1000000,7,16,0,20,32,34,22,0,394,56.378872
3,LB7408,5,5,1,33,^,^,1000000,8,18,0,23,38,38,29,0,397,64.794548
4,LB7408,5,5,1,34,^,^,1000000,9,19,0,27,40,44,30,0,401,73.878022


검증 Loop

- 월납 순보, 영보, 준비금 검증

- 오류 없이 100%까지 도달하면 검증 완료

In [8]:
for row in tqdm(df_check.values):
    # unpack
    cov, n, m, sex, x, injure, driver, AMT, NP12, G12, V0, V1, V2, V3, V4, V5, alpha_apply, alpha_std = row

    # set coverage key
    if injure !="^":
        covKey = f"{cov}_{injure}"
    elif driver != "^":
        covKey = f"{cov}_{driver}"
    else:
        covKey = cov

    # Calculation
    my_NP12, my_G12, my_tVx = pv.MyCalculation(sex, x, n, m, covKey, AMT, injure, driver)

    if NP12 != my_NP12:
        raise Exception(f'{covKey} 담보 순보험료 오류 | my : {my_NP12} | table : {NP12}')

    elif G12 != my_G12:
        raise Exception(f'{covKey} 담보 영업보험료 오류 | my : {my_G12} | table : {G12}')
        
    elif [int(V) for V in [V0,V1,V2,V3,V4,V5][:n]] != my_tVx[:n]:
        raise Exception(f'{covKey} 담보  준비금 오류')

100%|██████████| 16580/16580 [07:00<00:00, 39.41it/s]
